In [1]:
import pandas as pd
from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()


In [2]:
connection_string = "root:NewPassword@localhost:3306/election_prediction_db"
engine = create_engine(f'mysql://{connection_string}')


In [3]:
conn = engine.connect()

orig_election_dataset = pd.read_sql("Select * from All_2018_Finance_Demographics_VW", conn)

C:\Users\m_ron\Anaconda3\envs\PythonData\lib\site-packages\pymysql\cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, which will be replaced by UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


In [4]:
orig_election_dataset.head()

,st_abbrev,district,party,Total_Receipt,Total_Disbursement,Cash_On_Hand_COP,Debt_Owed_By_Committee,Individual_Itemized_Contribution,Individual_Unitemized_Contribution,Individual_Contribution,...,Management_of_companies_and_enterprises3,Admin_and_support_and_waste_mngmt_and_remediation_services3,Educational_services3,Health_care_and_social_assistance3,Arts__entertainment__and_recreation3,Accommodation_and_food_services3,Other_services_except_public_administration3,Industries_not_classified3,Industries_not_classified4,state_dist
0,AK,0,REP,1041380.0,1159940.0,151165.0,0.0,513967.0,41063.9,555031.0,...,193,1128,270,2633,569,2156,1823,40,None,AK_00
1,AL,1,REP,1325140.0,662806.0,1108650.0,0.0,264612.0,10105.0,274717.0,...,111,749,144,1427,198,1331,1663,21,None,AL_01
2,AL,2,REP,2506890.0,2100810.0,451536.0,0.0,741087.0,46129.8,787217.0,...,85,578,131,1538,159,1347,1673,29,None,AL_02
3,AL,3,REP,1219670.0,857764.0,887448.0,0.0,450890.0,9667.0,460557.0,...,47,521,89,1142,94,1186,1402,21,None,AL_03
4,AL,4,REP,1487010.0,853026.0,1099490.0,0.0,430365.0,23449.0,453814.0,...,76,401,58,1548,135,1060,1440,30,None,AL_04


In [5]:
orig_election_dataset = orig_election_dataset.fillna(0)

In [6]:
nc_index = int(orig_election_dataset[orig_election_dataset['state_dist']=='NC_09'].index[0])
print(nc_index)

245


In [7]:
orig_election_dataset = orig_election_dataset.drop([nc_index])


In [9]:
election_dataset = orig_election_dataset.drop(['id','st_abbrev', 'district', "Industries_not_classified4", "state_dist"], axis=1)

In [15]:
election_dataset.head()

,party,Total_Receipt,Total_Disbursement,Cash_On_Hand_COP,Debt_Owed_By_Committee,Individual_Itemized_Contribution,Individual_Unitemized_Contribution,Individual_Contribution,Other_Committee_Contribution,Party_Committee_Contribution,...,Real_estate_and_rental_and_leasing3,Professional__scientific__and_technical_services3,Management_of_companies_and_enterprises3,Admin_and_support_and_waste_mngmt_and_remediation_services3,Educational_services3,Health_care_and_social_assistance3,Arts__entertainment__and_recreation3,Accommodation_and_food_services3,Other_services_except_public_administration3,Industries_not_classified3
0,REP,1041380.0,1159940.0,151165.0,0.0,513967.0,41063.9,555031.0,481912.0,0.00,...,933,1960,193,1128,270,2633,569,2156,1823,40
1,REP,1325140.0,662806.0,1108650.0,0.0,264612.0,10105.0,274717.0,985650.0,663.98,...,846,1420,111,749,144,1427,198,1331,1663,21
2,REP,2506890.0,2100810.0,451536.0,0.0,741087.0,46129.8,787217.0,1628590.0,5000.00,...,561,1249,85,578,131,1538,159,1347,1673,29
3,REP,1219670.0,857764.0,887448.0,0.0,450890.0,9667.0,460557.0,742720.0,11500.00,...,462,853,47,521,89,1142,94,1186,1402,21
4,REP,1487010.0,853026.0,1099490.0,0.0,430365.0,23449.0,453814.0,1020840.0,0.00,...,385,808,76,401,58,1548,135,1060,1440,30


In [10]:
X = election_dataset.drop("party", axis=1)
y = election_dataset["party"]
print(X.shape, y.shape)

(434, 323) (434,)


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1, stratify=y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

C:\Users\m_ron\Anaconda3\envs\PythonData\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=500, activation='relu', input_dim=323))
model.add(Dense(units=500, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [20]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [21]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=2
)

Epoch 1/20
 - 1s - loss: 0.5732 - acc: 0.7631
Epoch 2/20
 - 0s - loss: 0.2081 - acc: 0.9200
Epoch 3/20
 - 0s - loss: 0.1301 - acc: 0.9692
Epoch 4/20
 - 0s - loss: 0.0740 - acc: 0.9785
Epoch 5/20
 - 0s - loss: 0.0370 - acc: 0.9908
Epoch 6/20
 - 0s - loss: 0.0159 - acc: 1.0000
Epoch 7/20
 - 0s - loss: 0.0089 - acc: 1.0000
Epoch 8/20
 - 0s - loss: 0.0061 - acc: 1.0000
Epoch 9/20
 - 0s - loss: 0.0043 - acc: 1.0000
Epoch 10/20
 - 0s - loss: 0.0033 - acc: 1.0000
Epoch 11/20
 - 0s - loss: 0.0026 - acc: 1.0000
Epoch 12/20
 - 0s - loss: 0.0021 - acc: 1.0000
Epoch 13/20
 - 0s - loss: 0.0018 - acc: 1.0000
Epoch 14/20
 - 0s - loss: 0.0015 - acc: 1.0000
Epoch 15/20
 - 0s - loss: 0.0013 - acc: 1.0000
Epoch 16/20
 - 0s - loss: 0.0011 - acc: 1.0000
Epoch 17/20
 - 0s - loss: 0.0010 - acc: 1.0000
Epoch 18/20
 - 0s - loss: 8.8729e-04 - acc: 1.0000
Epoch 19/20
 - 0s - loss: 8.0026e-04 - acc: 1.0000
Epoch 20/20
 - 0s - loss: 7.0254e-04 - acc: 1.0000
